# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM Schema

In almost all cases, rather than just working with a single table you will typically need data from multiple tables. 
Doing this requires the use of **joins** using shared columns from the two tables. 

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database **data.sqlite**.

In [1]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Display the names of all the employees in Boston.
Hint: join the employees and offices tables.

In [5]:
# employees.officeCode == offices.officeCode
cur.execute("""SELECT firstName, lastName 
               FROM employees 
               JOIN offices 
               USING(officeCode) 
               WHERE city = 'Boston';""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,firstName,lastName
0,Julie,Firrelli
1,Steve,Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by.

In [7]:
# employees.officeCode == offices.officeCode 
#if we don't have record about office in employees table then count office
cur.execute("""SELECT city, COUNT(*) as empty_offices
               FROM offices
               LEFT JOIN employees
               USING(officeCode)
               GROUP BY city;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()


,city,empty_office
0,Boston,2
1,London,2
2,NYC,2
3,Paris,5
4,San Francisco,6


## Write 3 Questions of your own and answer them

In [24]:
# Answers will vary
# Example: Display the htmlDescription and employee's first and last name 
#for each product that each employee has sold
cur.execute("""SELECT htmlDescription,e.firstName, e.lastName
               FROM employees e
               JOIN customers c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               JOIN orders o 
               USING(customerNumber)
               JOIN orderdetails od
               USING(orderNumber)
               JOIN products p
               USING(productCode)
               JOIN productlines pl
               USING(productLine)
               """)
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,htmlDescription,firstName,lastName
0,,Leslie,Jennings
1,,Leslie,Jennings
2,,Leslie,Jennings
3,,Leslie,Jennings
4,,Leslie,Jennings
...,...,...,...
2991,,Martin,Gerard
2992,,Martin,Gerard
2993,,Martin,Gerard
2994,,Martin,Gerard


In [32]:
#how much spent each customer 
cur.execute("""SELECT * FROM(SELECT sum(p.amount) as exp, c.contactLastName, e.lastName
               FROM employees e
               JOIN customers c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               JOIN payments p
               USING(customerNumber)
               GROUP BY e.lastName, c.contactLastName)
               WHERE exp > 100000
               ORDER BY exp,lastName
            """)
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,exp,contactLastName,lastName
0,101721.90,Young,Thompson
1,101872.52,Huang,Firrelli
2,103896.74,Suominen,Bott
3,104224.79,Bergulfsen,Jones
4,104545.22,Hashimoto,Jennings
5,105420.57,Victorino,Marsh
6,105548.73,Shimamura,Nishi
7,107446.50,Petersen,Castillo
8,107639.94,Lee,Vanauf
9,108777.92,Holz,Gerard


In [ ]:
cur.execute("""SELECT sum(p.amount), c.contactLastName, e.lastName
               FROM employees e
               JOIN customers c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               JOIN payments p
               USING(customerNumber)
               GROUP BY e.lastName, c.contactLastName
            """)
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

In [ ]:
# Your code here

## Level Up: Display the names of every individual product that each employee has sold

In [8]:
# Your code here
cur.execute("""SELECT firstName, lastName, productName
               FROM employees e
               JOIN customers c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               JOIN orders o
               USING(customerNumber)
               JOIN orderdetails od
               USING(orderNumber)
               JOIN products p
               USING(productCode)""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df))
df.head()

2996


,firstName,lastName,productName
0,Leslie,Jennings,1958 Setra Bus
1,Leslie,Jennings,1940 Ford Pickup Truck
2,Leslie,Jennings,1939 Cadillac Limousine
3,Leslie,Jennings,1996 Peterbilt 379 Stake Bed with Outrigger
4,Leslie,Jennings,1968 Ford Mustang


## Level Up: Display the Number of Products each employee has sold

In [9]:
# Your code here
cur.execute("""SELECT firstName, lastName, COUNT(productName)
               FROM employees e
               JOIN customers c                   
               ON e.employeeNumber = c.salesRepEmployeeNumber
               JOIN orders o 
               USING(customerNumber)
               JOIN orderdetails od 
               USING(orderNumber)                    
               JOIN products p 
               USING(productCode)
               GROUP BY lastName
               ORDER BY firstName""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df))
df


15


,firstName,lastName,COUNT(productName)
0,Andy,Fixter,185
1,Barry,Jones,220
2,Foon Yue,Tseng,142
3,George,Vanauf,211
4,Gerard,Hernandez,396
5,Julie,Firrelli,124
6,Larry,Bott,236
7,Leslie,Jennings,331
8,Leslie,Thompson,114
9,Loui,Bondur,177


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!